In [3]:
import pandas as pd 
import numpy as np 
import re 

In [4]:
np.logspace(-3,3,7)

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])

In [5]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/aamin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aamin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
data = pd.read_csv("/home/aamin/files/marketing/copy/reports_data/all_reports_data.csv")

/home/aamin/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6,8,13,14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
data.drop(["Unnamed: 0","Unnamed: 0.1"],axis = 1,inplace= True)

In [8]:
data.iloc[303690]

Broadcast ID                                                          3150
Broadcast name                                             DTSS Delhi 1211
Mobile number                                                   9702610491
Primary Key (User_id)                                             LOAN1672
Link clicked                                 https://mv343.app.goo.gl/3Pbh
Delivered Time                                         2021-11-12 15:21:22
Clicked time                                           2021-11-12 15:21:21
Delivered Date                                                  2021-11-12
Delivery Time                                                     15:21:22
Hour of Day                                                             15
Day of Week                                                         Friday
Clicked Date                                                    2021-11-12
Click Time                                                        15:21:21
Clicked Hour of Day      

In [9]:
# data.groupby(["Message","Broadcast name"]).size().to_frame()

In [10]:
data_copy = data[['Broadcast ID','Broadcast name', 'Mobile number',
       'Delivered Time','Delivered Date', 'Delivery Time', 'Hour of Day',
       'Day of Week','Clicked time', 'Clicked Date', 'Click Time', 'Clicked Hour of Day',
       'Message', 'Circle','SMS Language']]

In [11]:
data_copy['Num words in SMS'] = data_copy['Message'].apply(lambda x: len(x.split(' ')))

/tmp/ipykernel_10644/2989540575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy['Num words in SMS'] = data_copy['Message'].apply(lambda x: len(x.split(' ')))


In [12]:
# data_copy.drop(["Message"],axis = 1,inplace=True)

In [13]:
import time
from datetime import datetime
from dateutil import parser

def func_UTCToEpoch(utcData, timestampFormat=None):
    if timestampFormat == None:
        dateObj    = parser.parse(utcData)
        epochValue = dateObj.timestamp()
    else:
        timeTuple  = time.strptime(utcData, timestampFormat)
        epochValue = time.mktime(timeTuple)
    return (epochValue)

def calculate_timelag(delv_time, click_time):
    
    if click_time == click_time:
        delv_time_ts = datetime.fromtimestamp(func_UTCToEpoch(delv_time))
        click_time_ts = datetime.fromtimestamp(func_UTCToEpoch(click_time))
        diff = click_time_ts - delv_time_ts
        seconds = diff.days
    else:
        seconds = np.nan
    
    return seconds

In [14]:
data_copy['Timelag (days)'] = data_copy.apply(lambda x: calculate_timelag(x['Delivered Time'], x['Clicked time']), axis=1)

/tmp/ipykernel_10644/2648180225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy['Timelag (days)'] = data_copy.apply(lambda x: calculate_timelag(x['Delivered Time'], x['Clicked time']), axis=1)


In [18]:
data_copy.loc[(data_copy["Clicked time"].isna() == False),'label']=1
data_copy.loc[(data_copy["Clicked time"].isna() == True),'label']=0
data_copy[['label']] = data_copy[['label']].astype(np.int64)

/home/aamin/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/aamin/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/aamin/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [19]:
data_copy.columns

Index(['Broadcast ID', 'Broadcast name', 'Mobile number', 'Delivered Time',
       'Delivered Date', 'Delivery Time', 'Hour of Day', 'Day of Week',
       'Clicked time', 'Clicked Date', 'Click Time', 'Clicked Hour of Day',
       'Message', 'Circle', 'SMS Language', 'Num words in SMS',
       'Timelag (days)', 'label'],
      dtype='object')

In [20]:
data_copy_1 = data_copy[['Broadcast name','Mobile number',"Message",
       'Delivered Date', 'Day of Week','Hour of Day',
        'Clicked Date',
        'Circle', 'SMS Language', 'Num words in SMS',
       'Timelag (days)', 'label']].copy()

In [21]:
# data_copy[data_copy['Timelag (days)'] > 7]['Timelag (days)'].value_counts()
# data_copy_1[data_copy_1["SMS Language"] == "en"]["Message"]

In [22]:
# tie_up_data = data[data["Broadcast name"].str.contains("Tie" , flags= re.IGNORECASE)].copy()

In [23]:
# tie_up_data["Broadcast name"].value_counts()

In [24]:
broadcast_name = ["Cognizant" ,"Vinolya","UDS","ASPPL","Avtar","DTSS",
                  "Bluedart","Pragathi","DustBucket","Congnizant","pragarthi","Bangalore"]

rstr = '|'.join(broadcast_name)

data_copy_2 = data_copy_1[data_copy_1["Broadcast name"].str.contains(rstr, flags=re.IGNORECASE)]

In [25]:
data_copy_2["Broadcast name"].value_counts()

DTSS Delhi 1211                   40918
Tie Up DTSS Bang English 2511      9786
Tie Up DTSS Bang Hindi 2511        9786
Tie Up DTSS Bang English 0212      9767
Tie Up DTSS Bang Hindi 0212        9220
Tie Up UDS English 2411            5202
UDS Tie Up Hindi 2411              5202
UDS Tie Up Hindi 0512              5026
Tie Up UDS English 0512            5026
UDS Delhi 1311                     4935
UDS Delhi 1411                     4935
Cognizant 1105                     4343
Bangalore Tieups 1411              3911
Bangalore Tieups 1311              3911
DTSS Delhi 1411                    3223
DTSS Delhi 1311                    3223
Tie Up Dtss Delhi English 0512     2814
Tie Up DTSS Delhi Hindi 0512       2814
Tie Up DTSS Hindi 2411             2811
Tie Up Avtar DTSS 2411             2811
Tie Up Cognizant 1609              1560
Tie Up Pragarthi Hindi 2511         831
Tie Up Pragarthi English 2511       831
Tie Up Cognizant 0909               782
Tie Up Vinolya 1609                 781


In [61]:
# data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("dtss", flags= re.IGNORECASE) == True , "Circle"] = broadcast_name_data["Region"].iloc[9]

In [62]:
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("Bangalore", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("dtss hindi", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("dtss bang", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("Pragarthi", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("Pragathi", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("DustBucket", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("asppl", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("Vinolya", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("BlueDart", flags= re.IGNORECASE) == True , "Circle"] = "Karnataka"

data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("Cognizant", flags= re.IGNORECASE) == True , "Circle"] = "Hyderabad"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("Congnizant", flags= re.IGNORECASE) == True , "Circle"] = "Hyderabad"

data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("uds", flags= re.IGNORECASE) == True , "Circle"] = "Delhi"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("avtar", flags= re.IGNORECASE) == True , "Circle"] = "Delhi"
data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("dtss delhi", flags= re.IGNORECASE) == True , "Circle"] = "Delhi"


/home/aamin/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [63]:
# data_copy_2.loc[data_copy_2["Broadcast name"].str.contains("asppl", flags= re.IGNORECASE) == True , "Circle"]

In [64]:
# data_copy_2.groupby(["Circle","SMS Language","Broadcast name"]).size()

In [85]:
data_copy_2[(data_copy_2["Timelag (days)"] < 4) & 
            (data_copy_2["Timelag (days)"] > 1)]["SMS Language"].value_counts()

hi    40
en     6
kn     3
Name: SMS Language, dtype: int64

In [46]:
index_list = [1,2,3,4,5,6]
data_copy.loc[data_copy.index[index_list]]
    

,Broadcast ID,Broadcast name,Mobile number,Delivered Time,Delivered Date,Delivery Time,Hour of Day,Day of Week,Clicked time,Clicked Date,Click Time,Clicked Hour of Day,Message,Circle,SMS Language,Num words in SMS,Timelag (days),label
1,4862,Quess 3105,8920194583,2022-05-31 08:02:28,2022-05-31,08:02:28,8,Tuesday,2022-05-31 20:18:12,2022-05-31,20:18:12,20.0,"Dear Customer, Please download the MyShubhLIfe...",Madhya Pradesh-Chattisgarh,en,22,0.0,1
2,4862,Quess 3105,8860875024,2022-05-31 08:02:27,2022-05-31,08:02:27,8,Tuesday,2022-06-18 10:50:57,2022-06-18,10:50:57,10.0,"Dear Customer, Please download the MyShubhLIfe...",Delhi,en,22,18.0,1
3,5084,RNR Cases 3006,7095924021,2022-06-30 10:13:28,2022-06-30,10:13:28,10,Thursday,2022-06-30 10:13:38,2022-06-30,10:13:38,10.0,"Dear Customer, We noticed your interest in a M...",Tamil Nadu,en,29,0.0,1
4,5084,RNR Cases 3006,9494080943,2022-06-30 10:12:14,2022-06-30,10:12:14,10,Thursday,2022-06-30 10:12:19,2022-06-30,10:12:19,10.0,"Dear Customer, We noticed your interest in a M...",Chennai,en,29,0.0,1
5,5084,RNR Cases 3006,9284196934,2022-06-30 10:12:34,2022-06-30,10:12:34,10,Thursday,2022-06-30 10:12:37,2022-06-30,10:12:37,10.0,"Dear Customer, We noticed your interest in a M...",Delhi,en,29,0.0,1
6,5084,RNR Cases 3006,9987692267,2022-06-30 10:11:48,2022-06-30,10:11:48,10,Thursday,2022-06-30 10:11:49,2022-06-30,10:11:49,10.0,"Dear Customer, We noticed your interest in a M...",Mumbai,en,29,0.0,1


In [44]:
data_copy.loc[[2,4,5]]

,Broadcast ID,Broadcast name,Mobile number,Delivered Time,Delivered Date,Delivery Time,Hour of Day,Day of Week,Clicked time,Clicked Date,Click Time,Clicked Hour of Day,Message,Circle,SMS Language,Num words in SMS,Timelag (days),label
2,4862,Quess 3105,8860875024,2022-05-31 08:02:27,2022-05-31,08:02:27,8,Tuesday,2022-06-18 10:50:57,2022-06-18,10:50:57,10.0,"Dear Customer, Please download the MyShubhLIfe...",Delhi,en,22,18.0,1
4,5084,RNR Cases 3006,9494080943,2022-06-30 10:12:14,2022-06-30,10:12:14,10,Thursday,2022-06-30 10:12:19,2022-06-30,10:12:19,10.0,"Dear Customer, We noticed your interest in a M...",Chennai,en,29,0.0,1
5,5084,RNR Cases 3006,9284196934,2022-06-30 10:12:34,2022-06-30,10:12:34,10,Thursday,2022-06-30 10:12:37,2022-06-30,10:12:37,10.0,"Dear Customer, We noticed your interest in a M...",Delhi,en,29,0.0,1


In [78]:
data_copy_2[data_copy_2["Timelag (days)"] == -1]

,Broadcast name,Mobile number,Message,Delivered Date,Day of Week,Hour of Day,Clicked Date,Circle,SMS Language,Num words in SMS,Timelag (days),label,SN
8689,Cognizant 1105,6369446247,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,34
8699,Cognizant 1105,9848862026,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,44
8811,Cognizant 1105,9829972887,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,156
8841,Cognizant 1105,8285912266,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,186
8849,Cognizant 1105,8168189502,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,194
8889,Cognizant 1105,9824450893,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,234
8925,Cognizant 1105,7013069627,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,270
8953,Cognizant 1105,8851420746,"Dear Customer, Bring home the Happiness this M...",2022-05-11,Wednesday,16,2022-05-11,Hyderabad,en,32,-1.0,1,298
303690,DTSS Delhi 1211,9702610491,"प्रिय ग्राहक, बधाई हो! आपकी कंपनी हमारी विशिष्...",2021-11-12,Friday,15,2021-11-12,Delhi,hi,44,-1.0,1,7800
303886,DTSS Delhi 1211,8425068743,"प्रिय ग्राहक, बधाई हो! आपकी कंपनी हमारी विशिष्...",2021-11-12,Friday,15,2021-11-12,Delhi,hi,44,-1.0,1,7996


In [22]:
data_copy_2.drop(["Broadcast name"] , axis= 1, inplace=True)

/home/aamin/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [28]:
data_copy_2["Message"].value_counts()

Dear Customer, Congratulations! As an employee of our exclusive Shubh Saathi partner get attractive offers on our personal loans. Minimum take home salary is Rs.12000. Click here to apply now https://mv343.app.goo.gl/KRTr - Team MyShubhLife                                                    45586
प्रिय ग्राहक, बधाई हो! आपकी कंपनी हमारी विशिष्ट शुभ साथी पार्टनर है । इसलिए आप [BROKEN_PERIOD_INTEREST] फ्लैट की आकर्षक ब्याज दर पर MyShubhLife द्वारा ऋण के लिए पूर्व-अनुमोदित हैं। वेतन सीमा: [EMI_AMOUNT] तक। लाभ उठाने के लिए https://mv343.app.goo.gl/3Pbh पर क्लिक करें! - टीम MyShubhLife    40918
प्रिय ग्राहक, बधाई! आपकी कंपनी हमारा अनन्य शुभ साथी भागीदार ह। इसलिए आपको MyShubhLife द्वारा आकर्षक ब्याज दर पर ऋण के लिए पूर्व-अनुमोदित किया गया है। न्यूनतम वेतन आवश्यक रु.12,000. अभी लाभ उठाने के लिए https://mv343.app.goo.gl/KRTr क्लिक करें! - टीम MyShubhLife                               39994
प्रिय ग्राहक, बधाई! आपकी कंपनी हमारा अनन्य शुभ साथी भागीदार ह। इसलिए आपको MyShubhLife द्वारा आकर्षक ब्याज 

In [24]:
data_copy_2["SMS Language"].value_counts()

hi    97228
en    49929
kn     7822
Name: SMS Language, dtype: int64

In [29]:
data_copy_2["Message"].iloc[0]

'Dear Customer, Congratulations! As an employee of our exclusive Shubh Saathi partner get attractive offers on our personal loans. Minimum take home salary is Rs.12000. Click here to apply now https://mv343.app.goo.gl/KRTr - Team MyShubhLife'

In [74]:
data_copy_2.groupby(["Circle","SMS Language"]).size()

Circle     SMS Language
Delhi      en              16805
           hi              70853
Hyderabad  en               8244
           hi                779
Karnataka  en              24880
           hi              25596
           kn               7822
dtype: int64

In [39]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
  
stop_words = set(stopwords.words('english'))
  
word_tokens = word_tokenize(data_copy_2["Message"].iloc[0])
# converts the words in word_tokens to lower case and then checks whether 
# they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
# with no lower case conversion
filtered_sentence = []
  
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
  
print(word_tokens)
print(filtered_sentence)


# tokenizer = RegexpTokenizer(r'\w+')
# tokenizer.tokenize(data_copy_2["Message"].iloc[0])

['Dear', 'Customer', ',', 'Congratulations', '!', 'As', 'an', 'employee', 'of', 'our', 'exclusive', 'Shubh', 'Saathi', 'partner', 'get', 'attractive', 'offers', 'on', 'our', 'personal', 'loans', '.', 'Minimum', 'take', 'home', 'salary', 'is', 'Rs.12000', '.', 'Click', 'here', 'to', 'apply', 'now', 'https', ':', '//mv343.app.goo.gl/KRTr', '-', 'Team', 'MyShubhLife']
['Dear', 'Customer', ',', 'Congratulations', '!', 'As', 'employee', 'exclusive', 'Shubh', 'Saathi', 'partner', 'get', 'attractive', 'offers', 'personal', 'loans', '.', 'Minimum', 'take', 'home', 'salary', 'Rs.12000', '.', 'Click', 'apply', 'https', ':', '//mv343.app.goo.gl/KRTr', '-', 'Team', 'MyShubhLife']


In [70]:
data_copy_2['SN'] = np.arange(0,len(data_copy_2)).tolist()

/tmp/ipykernel_68685/3557631693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_copy_2['SN'] = np.arange(0,len(data_copy_2)).tolist()


In [71]:
data_copy_2.to_csv("individual_marketing_model_data.csv")